In [163]:
# Import necessary libraries
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [141]:
# ### Read CSV Files
# Load the training and testing datasets
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Limpando os dados

In [142]:
# ### Check for Duplicates
# Concatenate the training and testing datasets to check for duplicates
combined_data = pd.concat([train_data, test_data], ignore_index=True)

# Identify duplicated rows in the combined dataset
duplicates = combined_data.duplicated()

# Check if any duplicates exist and print the results
if duplicates.any():
    print("There are duplicated rows.")
    print(combined_data[duplicates])  # Show the duplicated rows
else:
    print("There are no duplicated rows.")

There are no duplicated rows.


In [143]:
# ### Handle Missing Values in 'Item_Weight'
# The 'Item_Weight' column has missing values that need to be addressed.
# Group by 'Item_Identifier' and calculate descriptive statistics to find a suitable replacement for missing values
weight_statistics = train_data.groupby('Item_Identifier').agg(
    mean=('Item_Weight', 'mean'),
    std=('Item_Weight', 'std'),
    min=('Item_Weight', 'min'),
    q25=('Item_Weight', lambda x: x.quantile(0.25)),
    q50=('Item_Weight', 'median'),
    q75=('Item_Weight', lambda x: x.quantile(0.75)),
    max=('Item_Weight', 'max')
)

# ### Remove Irrelevant Records
# Identify items with a single record that have a null weight
# Since there are only 4 such records in over 8k, they will be removed
records_to_remove = ['FDN52', 'FDK57', 'FDE52', 'FDQ60']
train_data = train_data[~train_data['Item_Identifier'].isin(records_to_remove)]

# ### Fill Missing Values with Means
# Replace null values in 'Item_Weight' with the mean weight of each item
mean_weight_per_item = train_data.groupby('Item_Identifier')['Item_Weight'].transform('mean')
train_data['Item_Weight'] = train_data['Item_Weight'].fillna(mean_weight_per_item)

In [144]:
# ### Standardize 'Item_Fat_Content' Categories
# Unify categories that represent the same concept
category_replacements = {
    'LF': 'Low Fat',
    'low fat': 'Low Fat',
    'reg': 'Regular'
}
train_data['Item_Fat_Content'] = train_data['Item_Fat_Content'].replace(category_replacements)

In [145]:
# ### Consolidate Rare Categories in 'Item_Type'
# Replace categories with fewer than 200 records as they may be underrepresented
train_data.loc[train_data['Item_Type'].isin(['Starchy Foods', 'Breakfast', 'Seafood']), 'Item_Type'] = 'Others'

In [146]:
# Existem items com visibilidade 0, parece extranho porém não irei editar. 
# O item pode não ficar exposto na loja, além disso esses itens são únicos por outlet então não teriam como substituir por alguma outra métrica

In [147]:
# Alguns Outlets não tem o valor do tamanho, ficará nulo mesmo pois não temos como saber o tamanho de fato do Outlet

# Tratando os dados

In [148]:
treated_train_data = train_data.copy()

In [149]:
# Padronização dos dados, apesar do XGB ser robusto com escalas diferentes, pretendo realizar uma analise de importancia das categorias e um tunning do modelo.
# A padronização pode ser benéfica para essas análises futuras. 
# Além disso, a utilizarei a Padronização pois ela é menos sensivel a outlier do que a Normalização
scaler = StandardScaler()
treated_train_data[['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']] = scaler.fit_transform(
    treated_train_data[['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']])

In [150]:
# Encoding nas colunas categoricas pois o XGBRegressor não aceita colunas categoricas

In [151]:
# Label Encoding na coluna de Gordura e Tamanho do outlet, visto que tem apenas 2 valores então essa binaridade será capturada
le_fat = LabelEncoder()
treated_train_data['Item_Fat_Content'] = le_fat.fit_transform(treated_train_data['Item_Fat_Content'])
treated_train_data['Outlet_Size'] = le_fat.fit_transform(treated_train_data['Outlet_Size'])

In [152]:
# One hot encoding no Item_Type pois tem muitas categorias e quero evitar induzir uma ordem/hierarquia nas categorias. O xgb funciona bem com entradas binárias
treated_train_data = pd.get_dummies(treated_train_data, columns=['Item_Type'], drop_first=True)

In [153]:
# Encoding na coluna Tier, apenas removi o "Tier " do nome da categoria
treated_train_data['Outlet_Location_Type'] = treated_train_data['Outlet_Location_Type'].str.replace('Tier ', '', regex=False).astype(int)

In [154]:
# One hot encoding no Outlet_Type pois tem muitas categorias e quero evitar induzir uma ordem/hierarquia nas categorias. O xgb funciona bem com entradas binárias
treated_train_data = pd.get_dummies(treated_train_data, columns=['Outlet_Type'], drop_first=True)

# Model

In [155]:
# Separando as categoria previsoras da que será prevista. 
# Removendo Identificadores pois geralmente não tem relação direta com as vendas e removendo a coluna q será prevista
x_train_data = treated_train_data.drop(['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'], axis = 1)
# Isolando a coluna q será prevista
y_train_data = treated_train_data['Item_Outlet_Sales']

In [160]:
# Treino do Modelo XGB
model = xgb.XGBRegressor()
model.fit(x_train_data, y_train_data, eval_set = [(x_train_data, y_train_data)])

[0]	validation_0-rmse:1413.11852
[1]	validation_0-rmse:1239.61964
[2]	validation_0-rmse:1142.26115
[3]	validation_0-rmse:1086.48732
[4]	validation_0-rmse:1054.40376
[5]	validation_0-rmse:1034.65314
[6]	validation_0-rmse:1019.07278
[7]	validation_0-rmse:1008.17237
[8]	validation_0-rmse:997.78937
[9]	validation_0-rmse:991.80089
[10]	validation_0-rmse:980.81496
[11]	validation_0-rmse:973.90219
[12]	validation_0-rmse:966.35763
[13]	validation_0-rmse:960.52228
[14]	validation_0-rmse:951.76886
[15]	validation_0-rmse:946.96109
[16]	validation_0-rmse:939.36103
[17]	validation_0-rmse:937.26896
[18]	validation_0-rmse:930.75491
[19]	validation_0-rmse:925.09663
[20]	validation_0-rmse:920.97174
[21]	validation_0-rmse:915.32086
[22]	validation_0-rmse:909.27095
[23]	validation_0-rmse:904.99484
[24]	validation_0-rmse:900.12461
[25]	validation_0-rmse:894.68683
[26]	validation_0-rmse:888.74248
[27]	validation_0-rmse:884.16218
[28]	validation_0-rmse:878.91794
[29]	validation_0-rmse:873.11050
[30]	validat

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [161]:
# Predição dos valores 
prediction = model.predict(x_train_data)

In [164]:
# Métricas do RMSE do modelo
rmse = np.sqrt(np.mean((y_train_data - prediction) ** 2))
rmse

651.0272790963623